# Robust Portfolio Optimization

The general goal is to select a *portfolio* of assets to maximize return, but we also take into account the "risk" of the portfolio. 
In general, the higher the return, the higher the risk. 
Our goal is to find portfolios that trade off return and risk in such a way as to make sure that, for a given risk appetite/tolerance, we find the highest return.

## Deterministic portfolio optimization (review of cvxpy)

To get warmed up, we'll model the classic Markowitz portfolio optmization problem. Given $n$ assets, with average returns $r$ and covariance matrix of returns $\Sigma$, we can express the problem of finding the minimum risk portfolio for a given return as
$$
\begin{array}{ll}
\text{minimize} & x^T \Sigma x\\
\text{subject to} & \mathbf{1}^T x = 1\\
& r^T x \ge R\\
& x \ge 0
\end{array}
$$

You can imagine a flipped problem where we maximize return subject to a risk tolerance. We also note that we've defined risk to be the variance of the resulting portfolio - this isn't probably a good idea in practice, but this is an older-style model.

### Setup imports and seed

In [9]:
pwd

'/Users/irina.wang/Desktop/Princeton/Project2/lropt/examples/portfolio'

In [10]:
import numpy as np
import cvxpy as cp
import lropt
np.random.seed(0)  # Reset seed

ModuleNotFoundError: No module named 'lropt'

### Generate portfolio data

In [2]:
def generate_data(n, N):
    """Generate synthetic portfolio data."""
    data = np.zeros((N, n))  # Preallocate date
    beta = [i / n for i in range(n)]  # Linking factors
    
    for sample_idx in range(N):
        # Common market factor, mean 3%, sd 5%, truncate at +- 3 sd
        mu, std = 0.03, 0.05
        z = np.random.normal(mu, std)
        z = np.minimum(np.maximum(z, mu - 3 * std), mu + 3 * std)
        
        for asset_idx in range(n):
            # Idiosyncratic contribution, mean 0%, sd 5%, truncated at +- 3 sd
            mu_id, std_id = 0.00, 0.05 
            asset = np.random.normal(mu_id, std_id)
            asset = np.minimum(np.maximum(asset, mu_id - 3 * std_id), mu_id + 3 * std_id)
            
            data[sample_idx, asset_idx] = beta[asset_idx] * z + asset
            
    return 100 * data

In [3]:
returns = generate_data(5, 10000)
print("Mean returns: ", np.round(np.mean(returns, 1), decimals=2))
print("Covariance returns: ", np.round(np.cov(returns), decimals=3))

Mean returns:  [ 9.24  4.85  4.91 ... -0.5   2.7  -2.5 ]
Covariance returns:  [[ 43.658   3.415  20.09  ...  -5.64    7.607   3.32 ]
 [  3.415  30.174   7.063 ...   4.838   0.282   9.323]
 [ 20.09    7.063  16.23  ...  -1.646  15.151  -9.429]
 ...
 [ -5.64    4.838  -1.646 ...   2.088  -2.193  -0.303]
 [  7.607   0.282  15.151 ...  -2.193  27.183 -20.286]
 [  3.32    9.323  -9.429 ...  -0.303 -20.286  35.024]]


### Evaluate portfolio selection

In [4]:
def evaluate_portfolio(returns, x):
    returns_x = returns.dot(x)
    returns_x.sort()
    n = len(returns_x)
    # Collect summary statistics for returns
    print(" minimum: %6.3f" % returns_x[0])
    print(" maximum: %6.3f" % returns_x[-1])
    print(" mean:    %6.3f" % returns_x.mean())
    print(" std dev: %6.3f" % returns_x.std())
    print(" maximum: %6.3f" % returns_x.var())

In [5]:
print("Stats fora portfolio investing all equally")
evaluate_portfolio(returns, 0.2 * np.ones(5))

Stats fora portfolio investing all equally
 minimum: -9.943
 maximum: 12.373
 mean:     1.171
 std dev:  2.987
 maximum:  8.925


### Solve deterministic portfolio

In [26]:
def solve_deterministic_portfolio(returns, min_return):
    # Get data
    r = np.mean(returns, 0)
    Sigma = np.cov(returns.T)
    n = returns.shape[1]
    
    # Define and solve problem
    x = cp.Variable(n)
    constraints = [0 <= x, x <= 1, cp.sum(x) == 1]
#     constraints += [r * x >= min_return]
#     objective = cp.Minimize(cp.quad_form(x, Sigma))
    cp.Problem(objective, constraints).solve(verbose=True)
    return x.value

In [27]:
train_returns = generate_data(10, 10000)
x_deterministic = solve_deterministic_portfolio(train_returns, 1.216)
print("Portfolio: ", np.round(x_deterministic, decimals=2))
evaluate_portfolio(train_returns, x_deterministic)

-----------------------------------------------------------------
           OSQP v0.5.0  -  Operator Splitting QP Solver
              (c) Bartolomeo Stellato,  Goran Banjac
        University of Oxford  -  Stanford University 2019
-----------------------------------------------------------------
problem:  variables n = 10, constraints m = 22
          nnz(P) + nnz(A) = 95
settings: linear system solver = qdldl,
          eps_abs = 1.0e-04, eps_rel = 1.0e-04,
          eps_prim_inf = 1.0e-04, eps_dual_inf = 1.0e-04,
          rho = 1.00e-01 (adaptive),
          sigma = 1.00e-06, alpha = 1.60, max_iter = 10000
          check_termination: on (interval 25),
          scaling: on, scaled_termination: off
          warm start: on, polish: on

iter   objective    pri res    dua res    rho        time
   1   0.0000e+00   1.22e+00   4.83e+03   1.00e-01   1.28e-03s
  75   6.2339e+00   1.20e-13   3.40e-11   5.52e-01   2.40e-03s
plsh   6.2339e+00   2.56e-16   1.20e-14   --------   2.51e-03s

s

## Robust portfolio optimization
In a robust portfolio optmization problem, we still have our $n$ assets and our goal is to balance risk and return. Instead of using the mean and variance of the portfolio, we'll instead try to maximize the worst-case return over our uncertainty set, i.e.,
$$
\begin{array}{ll}
\text{minimize} &\text{max}_{r \in U}  r^T x\\
\text{subject to} & \mathbf{1}^T x = 1\\
& x \ge 0
\end{array}
$$
The tradeoff between risk and return is baked into the uncertainty set $U$.

### Box uncertainty set
Given past returns, calculate average return $\bar{r}$ and the standard deviation of return $\sigma$ for each asset. Then our uncertainty set is
$$
U = \left\{ \bar{r} - \rho*\sigma \leq r \leq \bar{r} + \rho*\sigma \right\}
$$
where $\rho$ controls the size of the box. If $\rho=0$, $r$ is just the average return.

In [22]:
def solve_portfolio_box(returns, alpha):
    n = returns.shape[1]
    
    # Uncertainty
    r_bar = returns.mean(axis=0)
    sigma = returns.std(axis=0)
    r = lropt.UncertainParameter(n, uncertainty_set=lropt.Box(center=r_bar,
                                                            side=2 * sigma,
                                                            rho=alpha))

    x = cp.Variable(n)
    constraints = [cp.sum(x) == 1,
                   x >= 0]
    
    obj = cp.Variable()  # Hack to add the objective
    constraints += [r * x >= obj]
    
    objective = cp.Maximize(obj)
    problem = lropt.RobustProblem(objective, constraints)
    print(problem)
    problem.solve()
    
    return x.value

In [25]:
train_returns = generate_data(10, 10000)
x_portfolio_box = solve_portfolio_box(train_returns, 0.001)
print("Portfolio: ", np.round(x_portfolio_box, decimals=2))
evaluate_portfolio(train_returns, x_portfolio_box)

maximize var1132
subject to Sum(var1122, None, False) == 1.0
           0.0 <= var1122
           var1132 <= param1121 * var1122
Portfolio:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 minimum: -20.032
 maximum: 27.820
 mean:     2.688
 std dev:  6.662
 maximum: 44.379
